<a href="https://colab.research.google.com/github/AlecJJ/Stores/blob/main/Alexnet_Store_sort_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modules needed for code to function

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from google.colab import drive

Loading Google Drive folders

In [ ]:
# 1. Mount Google Drive
drive.mount('/content/gdrive')

# 2. Load labeled images from folders 

#   Data directory
data_dir = '/content/gdrive/MyDrive/Data2023/Stores'

Mounted at /content/gdrive


Pre-processing training and valid data

In [ ]:
# 3. Pre-process the data and create data loaders
# 80% training and 20% valid
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


Dataloader

In [ ]:
image_datasets = {x: datasets.ImageFolder(data_dir + '/' + x, data_transforms[x]) for x in ['train', 'valid']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'valid']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Alexnet architecture

In [ ]:
# 4. Set up the AlexNet architecture
alexnet = models.alexnet(pretrained=True)
num_ftrs = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_ftrs, len(class_names))
alexnet = alexnet.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 225MB/s]


Alexnet model training

In [ ]:


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)

# 5. Train the AlexNet model
num_epochs = 10

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    for phase in ['train', 'valid']:
        if phase == 'train':
            alexnet.train()
        else:
            alexnet.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = alexnet(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

    print()

print('Training complete')

# Save the trained model
torch.save(alexnet.state_dict(), '/content/gdrive/MyDrive/alexnet_classification.pth')



Epoch 0/9
----------
train Loss: 0.9622 Acc: 0.5128
valid Loss: 0.6152 Acc: 0.5455

Epoch 1/9
----------
train Loss: 0.6999 Acc: 0.5641
valid Loss: 0.5609 Acc: 0.7273

Epoch 2/9
----------
train Loss: 0.4701 Acc: 0.7692
valid Loss: 0.4357 Acc: 0.7273

Epoch 3/9
----------
train Loss: 0.4516 Acc: 0.7949
valid Loss: 0.7399 Acc: 0.7273

Epoch 4/9
----------
train Loss: 0.5598 Acc: 0.6923
valid Loss: 0.8850 Acc: 0.7273

Epoch 5/9
----------
train Loss: 0.4140 Acc: 0.7692
valid Loss: 0.3341 Acc: 0.7273

Epoch 6/9
----------
train Loss: 0.5597 Acc: 0.7179
valid Loss: 0.4018 Acc: 0.7273

Epoch 7/9
----------
train Loss: 0.3898 Acc: 0.8205
valid Loss: 0.7179 Acc: 0.7273

Epoch 8/9
----------
train Loss: 0.3107 Acc: 0.8462
valid Loss: 0.6459 Acc: 0.7273

Epoch 9/9
----------
train Loss: 0.3239 Acc: 0.8974
valid Loss: 0.3340 Acc: 0.8182

Training complete


Load trained Alexnet model and predicted results

In [ ]:
import requests
from PIL import Image
from io import BytesIO

# 1. Load the trained AlexNet model
def load_model(model_path):
    model = models.alexnet()
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, len(class_names))
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model.to(device)

model_path = '/content/gdrive/MyDrive/alexnet_classification.pth'
trained_model = load_model(model_path)

# 2. Define a function to load an image from a URL and preprocess it
def preprocess_image(url, transform):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    img_tensor = transform(img)
    return img_tensor.unsqueeze(0).to(device)

# 3. Perform inference using the loaded model
def predict_image_url(url, model, transform):
    img_tensor = preprocess_image(url, transform)
    with torch.no_grad():
        output = model(img_tensor)
        _, pred = torch.max(output, 1)
    return class_names[pred]

image_url = 'https://static.dw.com/image/46424739_905.jpg'
prediction = predict_image_url(image_url, trained_model, data_transforms['valid'])

print("The predicted class for the input image is:", prediction)


The predicted class for the input image is: Most crowded
